# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [1]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [2]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [3]:
api = morpc.census.api_get(url, params)

Total variables requested: 3
Starting request #1. 3 variables remain.


In [4]:
api

,NAME,B01001_001E
GEO_ID,,
0500000US39041,"Delaware County, Ohio",226296
0500000US39049,"Franklin County, Ohio",1321820


## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [5]:
acs = morpc.census.acs_data('B11001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [6]:
acs.VARS

{'B11001_001E': {'label': 'Estimate!!Total:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_001EA,B11001_001M,B11001_001MA'},
 'B11001_002E': {'label': 'Estimate!!Total:!!Family households:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_002EA,B11001_002M,B11001_002MA'},
 'B11001_003E': {'label': 'Estimate!!Total:!!Family households:!!Married-couple family',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_003EA,B11001_003M,B11001_003MA'},
 'B11001_004E': {'label': 'Estimate!!Total:!!Family households:!!Other family:',
  'concept': 'Household Type (Including Living Alone)',
  'predicateType': 'int',
  'group': 'B11001',
  'limit': 0,
  'attributes': 'B11001_004EA,B11001_004M,B11001_004MA'},
 'B11001_005E': {'

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [7]:
morpc.census.SCOPES

{'us-states': {'desc': 'all states in the United States',
  'for': 'state:*',
  'in': 'us:*'},
 'ohio': {'desc': 'the State of Ohio', 'for': 'state:39'},
 'ohio-counties': {'desc': 'all counties in the State of Ohio',
  'for': 'county:*',
  'in': 'state:39'},
 'ohio-tracts': {'desc': 'all Census tracts in the State of Ohio',
  'for': 'tracts:*',
  'in': 'state:39'},
 'region15-counties': {'desc': 'all counties in the MORPC 15-county region',
  'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141',
  'in': 'state:39'},
 'region15-tracts': {'desc': 'all Census tracts in the MORPC 10-county region',
  'for': 'tracts:*',
  'in': ['state:39',
   'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141']},
 'regionmpo-parts': {'desc': 'all Census township parts and place parts that are MORPC MPO members',
  'ucgid': '1550000US3902582041,0700000US390410577499999,0700000US390410578899999,0700000US390410942899999,1550000US3918000041,0700000US390411814099999,1550

In [10]:
acs = acs.query(scope='region15-counties')

morpc-acs5-2023-region15-counties-b11001 schema is valid
Total variables requested: 19
Starting request #1. 19 variables remain.


In [13]:
data = acs.DATA
data.head()

,B11001_001E,B11001_001M,B11001_002E,B11001_002M,B11001_003E,B11001_003M,B11001_004E,B11001_004M,B11001_005E,B11001_005M,B11001_006E,B11001_006M,B11001_007E,B11001_007M,B11001_008E,B11001_008M,B11001_009E,B11001_009M
GEO_ID,,,,,,,,,,,,,,,,,,
0500000US39041,80455,530,61028,838,51755,1091,9273,752,3127,446,6146,586,19427,950,16217,917,3210,400
0500000US39045,59979,510,43253,884,33336,923,9917,792,3221,424,6696,662,16726,795,13678,805,3048,454
0500000US39047,11635,239,7341,347,5388,325,1953,260,693,161,1260,206,4294,372,3425,327,869,206
0500000US39049,547922,1767,312682,3140,210814,2936,101868,2740,27792,1453,74076,2069,235240,3220,181656,3175,53584,1904
0500000US39073,11565,286,8201,369,5923,417,2278,343,952,279,1326,257,3364,345,2816,329,548,120


### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

In [15]:
acs = acs.query(get_param=['B11001_001E'], for_param='state:39')

morpc-acs5-2023-custom-ucgid-b11001-20250703-145508 schema is valid
Total variables requested: 3
Starting request #1. 3 variables remain.


In [16]:
acs.DATA

,B11001_001E,B11001_001M
GEO_ID,,
0400000US39,4829571,8204


### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [26]:
acs = morpc.census.acs_data('B11001', '2023', '5')
acs = acs.query(scope='region15-counties', get_param=['B11001_002E', 'B11001_007E'])
acs.DIM_TABLE

morpc-acs5-2023-region15-counties-b11001 schema is valid
Total variables requested: 5
Starting request #1. 5 variables remain.


,GEO_ID,VARIABLE,VALUE,VAR_TYPE,DIM_1
0,0500000US39041,B11001_002E,61028,Estimate,Family households
1,0500000US39045,B11001_002E,43253,Estimate,Family households
2,0500000US39047,B11001_002E,7341,Estimate,Family households
3,0500000US39049,B11001_002E,312682,Estimate,Family households
4,0500000US39073,B11001_002E,8201,Estimate,Family households
5,0500000US39083,B11001_002E,15762,Estimate,Family households
6,0500000US39089,B11001_002E,47574,Estimate,Family households
7,0500000US39091,B11001_002E,13049,Estimate,Family households
8,0500000US39097,B11001_002E,10139,Estimate,Family households
9,0500000US39101,B11001_002E,15263,Estimate,Family households


### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [27]:
acs.SCHEMA

{'fields': [{'name': 'GEO_ID',
             'type': 'string',
             'description': 'Unique identifier for geography'},
            {'name': 'B11001_002E',
             'type': 'integer',
             'description': 'Estimate!!Total:!!Family households: | Household '
                            'Type (Including Living Alone) | Estimate'},
            {'name': 'B11001_002M',
             'type': 'integer',
             'description': 'Estimate!!Total:!!Family households: | Household '
                            'Type (Including Living Alone) | MOE'},
            {'name': 'B11001_007E',
             'type': 'integer',
             'description': 'Estimate!!Total:!!Nonfamily households: | '
                            'Household Type (Including Living Alone) | '
                            'Estimate'},
            {'name': 'B11001_007M',
             'type': 'integer',
             'description': 'Estimate!!Total:!!Nonfamily households: | '
                            'Household Ty

In [28]:
acs.save(output_dir='./temp_data/')

Resource is valid.


In [29]:
acs.RESOURCE

{'name': 'morpc-acs5-2023-region15-counties-b11001',
 'type': 'table',
 'title': '2023 American Community Survey 5-Year Estimates for all counties in '
          'the MORPC 15-county region.',
 'description': 'Selected variables from 2023 ACS 5-Year estimates for all '
                'counties in the MORPC 15-county region. Data was retrieved '
                '2025-07-03',
 'profile': 'tabular-data-resource',
 'sources': [{'title': '2023 American Community Survey 5-Year Estimates, U.S. '
                       'Census Bureau',
              'path': 'https://api.census.gov/data/2023/acs/acs5',
              '_params': {'get': 'GEO_ID,B11001_002E,B11001_002M,B11001_007E,B11001_007M',
                          'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141',
                          'in': 'state:39'}}],
 'path': 'morpc-acs5-2023-region15-counties-b11001.csv',
 'scheme': 'file',
 'format': 'csv',
 'mediatype': 'text/csv',
 'encoding': 'utf-8',
 'hash': 'fd73f

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()